In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
#from  pandasgui import show
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import cross_val_predict
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
train_data = pd.read_csv('../data/airline_passenger_satisfaction.csv')
print(train_data.info())
primeros_100_registros = train_data.iloc[:100]
test_data = pd.DataFrame(primeros_100_registros)
primer_registro = train_data.iloc[0]
test_data = pd.DataFrame([primer_registro])
print(train_data.info())

In [ ]:
#obtenemos un arreglo con los nombres de las variables segun su tipo
imputer_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in train_data.columns if train_data[cname].dtype == "object"]

In [ ]:
imputer = SimpleImputer(strategy='mean' )
imputer.fit(train_data[imputer_cols])
train_data[imputer_cols] = imputer.transform(train_data[imputer_cols])
test_data[imputer_cols] = imputer.transform(test_data[imputer_cols])

In [ ]:
# completamos valores nulos  en las columnas categoricas con la moda
def fill_null_with_mode(column, train_df):#test_df
    moda = train_df[column].mode().iloc[0]
    train_df[column] = train_df[column].fillna(moda)
    #test_df[column] = test_df[column].fillna(moda)

# Aplicar la función de llenado de valores nulos
fill_null_with_mode(categorical_cols, train_data)#test_data

In [ ]:
#preparamos los datos para dividirlos
train_data.drop(["Unnamed: 0" ,"id"] , axis = 1 ,inplace = True)
test_data.drop(["Unnamed: 0" ,"id","satisfaction"] , axis = 1 ,inplace = True)
# TODO no estoy seguro si debo elimiar satisfaction de test

In [ ]:
# Crear una copia de los datos originales para evitar modificarlos
X = train_data.drop("satisfaction", axis=1)
y = train_data["satisfaction"]

In [ ]:


# Lista de columnas numéricas
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Lista de columnas categóricas en X
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]

# Escalar las características numéricas
numerical_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])

# Codificar one-hot las características categóricas
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse=False))])

# Combinar transformaciones para X
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Aplicar las transformaciones a X
X = preprocessor.fit_transform(X)

# Imprimir nueva forma de X
print('Training set shape (X):', X.shape)

#### Grid Search

In [ ]:
param_grid = {
    'n_estimators': range(20, 101, 20),
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'criterion' : ['gini', 'entropy']
}

scoring = 'accuracy'

clf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=scoring, cv=2, n_jobs=1)

In [ ]:
# Fit the grid search object to the training data
grid_search.fit(X, y)
 
# Print the best hyperparameters and the best score
print("Best hyperparameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)
 
# Re-train the model with the best hyperparameters
best_clf = grid_search.best_estimator_
best_clf.fit(X, y)
 
# Test the model with the best hyperparameters on the testing data
accuracy = best_clf.score(X, y)
print("Testing accuracy:", accuracy)

#### Validacion cruzada

In [ ]:
# Crear el pipeline con los parametros del grid search
my_pipeline = Pipeline(steps=[
    ('model', RandomForestClassifier())
])

my_pipeline.named_steps['model'].set_params(**grid_search.best_params_)

# Realizar la validación cruzada y obtener las probabilidades y los scores
proba_predictions = cross_val_predict(my_pipeline, X, y, cv=10, method='predict_proba')
accuracy_scores = cross_val_predict(my_pipeline, X, y, cv=10, method='predict')

In [ ]:
# Calcular promedio de las probabilidades de la clase positiva
preds = proba_predictions[:, 1].mean()

# Codificar las etiquetas de clase en un formato binario
lb = LabelBinarizer()
y_bin = lb.fit_transform(y)

# Calcular matriz de confusión
confusion = confusion_matrix(y, accuracy_scores)

# Calcular precisión
precision = precision_score(y, accuracy_scores, pos_label='satisfied')

# Calcular recuperación
recall = recall_score(y, accuracy_scores, pos_label='satisfied')

# Calcular puntaje F1
f1 = f1_score(y, accuracy_scores, pos_label='satisfied')

# Calcular AUC-ROC
roc_auc = roc_auc_score(y, proba_predictions[:, 1])

# Calcular AUC-PR
pr_auc = average_precision_score(y_bin, proba_predictions[:, 1])  # Usar y_bin

# Imprimir los resultados
print("Average probability:", preds)
print("Matriz de Confusión:")
print(confusion)
print("Precisión:", precision)
print("Recuperación:", recall)
print("Puntaje F1:", f1)
print("AUC-ROC:", roc_auc)
print("AUC-PR:", pr_auc)

#### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,train_size=0.8,test_size=0.2,random_state=0)

In [ ]:
classifier = RandomForestClassifier(criterion = 'gini', max_depth= None, min_samples_split=2, n_estimators=100)
classifier.fit(X_train,y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
# Calcular promedio de las probabilidades de la clase positiva
#preds = proba_predictions[:, 1].mean()

# Calcular promedio del score de precisión
average_accuracy = accuracy_score(y_test, y_pred)

#Matriz
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcula la precisión
precision = precision_score(y_test, y_pred, average='macro')

# Calcula el recall
recall = recall_score(y_test, y_pred, average='macro')

# Calcula el F1-Score
f1 = f1_score(y_test, y_pred, average='macro')

# Imprimir los resultados
#print("Average probability:", preds)
print("Average accuracy:", average_accuracy)
print("Matriz de Confusión:\n", conf_matrix)
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


**Average probability**  
Esto nos indica que de la cantidad de clientes satisfechos (satisfied) hay un 43.35% del total general , por lo tanto lo restante serian los que estan como "neutral or dissatisfied".

**Average accuracy**  
Aqui podemos observar que en cuanto a predicciones correctas, es decir que nuestro modelo cuenta con un 96.15% de acertividad en cuanto a las pruebas realizadas.

**Matriz de confusión**  
La matriz de confusión es la variable que nos permite determinar la capacidad del modelo para evitar clasificar incorrectamente las instancias negativas como positivas. A diferencia de la precisión uqe nos da una vision general de la acertividad , la matriz de confusión se utiliza para saber la categorizacion de las predicciones.

En esta métrica nos indica cuantos verdaderos positivos hay (11510) es decir que la mayoria de resultados aqui estan estado "satisfied"; Falsos positivos aqui se puede ver que el modelo interpretó 266 clientes que en realidad estan en estado  "neutral or dissatisfied" pero que el modelo tomó como "satisfied"; En cuanto a 579 , nos indica que el modelo interpreta estos datos como falsos negativos , que quiere decir que en realidad estos clientes estan como "satisfied" pero el modelo los tomó como "neutral or dissatisfied"; por último 8426 que significa que aqui estan los verdaderos negativos , que aqui todos los clientes estan como "neutral or dissatisfied".

**Precisión**  
Esto nos indica que de toda la base de datos el 96% es acertado con respecto a predicciones positivas, a diferencia de ka atriz de confusión , la precisión no tiene en cuenta estos falsos positivos y se centra en la proporción de predicciones positivas correctas en relación con todas las predicciones positivas. 

**Recall**  
El recall responde a la pregunta: "De todos los casos positivos reales, ¿cuántos de ellos el modelo fue capaz de identificar correctamente?". Es una métrica importante en problemas donde la detección de todos los casos positivos es crítica, como en la detección temprana de enfermedades o la identificación de fraudes. En este caso se puede determinar que el 96% de de los casos verdaderos positivos es correctamente identificado por el modelo.

**F1-Score**  
El F1-Score es útil cuando deseamos tener un equilibrio entre la precisión y el recall. Proporciona una puntuación que combina ambas métricas y es útil cuando ninguna de las dos métricas por sí sola es suficiente. En nuestro caso indica que está cerca al 1.0 entonces indica un buen equilibrio entre la precisión y el recall.


In [ ]:
# Obtiene la importancia de las características
importancia_caracteristicas = classifier.feature_importances_

# Puedes imprimir la importancia de cada característica
for i, importancia in enumerate(importancia_caracteristicas):
    print(f'Característica {i}: {importancia}')

In [ ]:
# Obtén las probabilidades de predicción del modelo (ya calculadas durante la validación cruzada)
probs = proba_predictions  # No necesitas llamar a predict_proba

# Crea una lista binaria de etiquetas, 1 para "satisfied" y 0 para "neutral or dissatisfied"
y_true_binary = [1 if label == "satisfied" else 0 for label in y]

# Calcula la curva ROC especificando pos_label=1
fpr, tpr, umbrales_roc = roc_curve(y_true_binary, probs[:, 1], pos_label=1)

# Calcula el área bajo la curva ROC (AUC-ROC)
roc_auc = auc(fpr, tpr)

# Calcula la curva PR
precision, recall, umbrales_pr = precision_recall_curve(y_true_binary, probs[:, 1])

# Calcula el área bajo la curva PR (AUC-PR)
pr_auc = auc(recall, precision)

# Graficar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

# Graficar la curva PR
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='blue', lw=2, label='Curva PR (AUC = %0.2f)' % pr_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recuperación')
plt.ylabel('Precisión')
plt.title('Curva de Precisión-Recuperación')
plt.legend(loc='lower left')
plt.show()

#### Analisis de curva ROC 

En la curva de ROC podremos notar como el modelo tiene un 0.99 de acertividad a la hora de clasificar, sabiendo distinguir correctamente entre valores correctos e incorrectos con respecto al humbral. La curva de satisfechos y la curva de no satisfechos no se superponen casi en lo absoluto, permitiendo al modelo una facil identificacion y teniendo un minimo error al distinguir entre uno o el otro. 

## Tecnicas ensemble

### Gradient boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

my_pipeline = Pipeline(steps=[
    ('model', GradientBoostingClassifier(n_estimators=50))
])

# Validación cruzada
proba_predictions = cross_val_predict(my_pipeline, X, y, cv=10, method='predict_proba')
accuracy_scores = cross_val_predict(my_pipeline, X, y, cv=10, method='predict')

# Calcular promedio de las probabilidades de la clase positiva
preds = proba_predictions[:, 1].mean()

# Precisión
average_accuracy = accuracy_score(y, accuracy_scores)

print("Average probability:", preds)
print("Average accuracy:", average_accuracy)


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

my_pipeline = Pipeline(steps=[
    ('model', AdaBoostClassifier(n_estimators=50))
])

# Validación cruzada
proba_predictions = cross_val_predict(my_pipeline, X, y, cv=10, method='predict_proba')
accuracy_scores = cross_val_predict(my_pipeline, X, y, cv=10, method='predict')

# Promedio de las probabilidades de la clase positiva
preds = proba_predictions[:, 1].mean()

# Precisión
average_accuracy = accuracy_score(y, accuracy_scores)

print("Average probability:", preds)
print("Average accuracy:", average_accuracy)


### Modelo ensemble Stackin utilizando ramdom Forest, Gradient Boosting y AdaBoost

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, train_size=0.8, test_size=0.2, random_state=0)

# Entrenando los tres modelos seleccionados
random_forest = RandomForestClassifier(n_estimators=50)
adaboost = AdaBoostClassifier(n_estimators=50)
gradient_boosting = GradientBoostingClassifier(n_estimators=50)

# Modelo de nivel superior (meta-modelo)
meta_classifier = RandomForestClassifier(n_estimators=50)  # Puedes cambiar esto según tus necesidades

models = [
    ('random_forest', random_forest),
    ('adaboost', adaboost),
    ('gradient_boosting', gradient_boosting)
]

# Modelo de stacking
stacking_model = StackingClassifier(estimators=models, final_estimator=meta_classifier)

# Entreamiento del modelo stacking
stacking_model.fit(X_train, y_train)

# Predicciones
y_pred = stacking_model.predict(X_test)

# Precisión
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:\n", conf_matrix)

# precisión (Precision)
precision = precision_score(y_test, y_pred, average='macro')
print("Precisión (Macro):", precision)

# Recall
recall = recall_score(y_test, y_pred, average='macro')
print("Recall (Macro):", recall)

# F1-Score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1-Score (Macro):", f1)


In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np

# Define un rango de tamaños de conjunto de entrenamiento
train_sizes, train_scores, test_scores = learning_curve(my_pipeline, X, y, cv=10, train_sizes=np.linspace(0.1, 1.0, 10))

# Calcula las medias y desviaciones estándar de las puntuaciones para el entrenamiento y prueba
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

# Grafica las curvas de aprendizaje
plt.figure(figsize=(10, 6))
plt.title("Curvas de Aprendizaje")
plt.xlabel("Tamaño del Conjunto de Entrenamiento")
plt.ylabel("Puntuación (Accuracy)")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Entrenamiento")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Prueba")

plt.legend(loc="best")
plt.show()

In [ ]:
# Calcula la diferencia entre las puntuaciones de entrenamiento y prueba para cada tamaño de conjunto de entrenamiento
overfitting_measure = train_scores_mean - test_scores_mean

# Grafica la medida de overfitting
plt.figure(figsize=(10, 6))
plt.title("Medida de Overfitting")
plt.xlabel("Tamaño del Conjunto de Entrenamiento")
plt.ylabel("Diferencia de Puntuaciones (Entrenamiento - Prueba)")
plt.grid()

plt.plot(train_sizes, overfitting_measure, 'o-', color="b", label="Overfitting Measure")

plt.legend(loc="best")
plt.axhline(y=0, color='r', linestyle='--')  # Línea de referencia en 0

plt.show()

# Imprime la medida de overfitting para el último tamaño de conjunto de entrenamiento
print("Medida de Overfitting para el último tamaño de conjunto de entrenamiento:", overfitting_measure[-1])


In [ ]:
from sklearn.metrics import accuracy_score

# Entrena tu modelo en el conjunto de entrenamiento
classifier.fit(X_train, y_train)

# Realiza predicciones en el conjunto de entrenamiento y prueba
y_train_pred = classifier.predict(X_train)
y_test_pred = classifier.predict(X_test)

# Calcula la precisión en el conjunto de entrenamiento y prueba
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Calcula el overfitting como la diferencia porcentual entre la precisión en entrenamiento y prueba
overfitting_percentage = (train_accuracy - test_accuracy) * 100

print(f"Overfitting Percentage: {overfitting_percentage:.2f}%")
